# Backtesting a Strategy

In [ ]:
!pip install import_ipynb --quiet
!pip install anvil-uplink --quiet
!pip install yfinance --quiet
!pip install pandas_ta --quiet
!pip install ipynb --quiet
!pip install rpyc --quiet

In [ ]:
!pip install -q --upgrade --force-reinstall --no-deps kaggle

In [1]:
import pandas as pd
import numpy as np
import import_ipynb
from backtest import Backtest
from feeds import BackFeed,DataFeed
from validation import Validate

importing Jupyter notebook from backtest.ipynb
importing Jupyter notebook from feeds.ipynb
importing Jupyter notebook from featfuncs.ipynb
importing Jupyter notebook from india_calendar.ipynb
importing Jupyter notebook from feed_env.ipynb
importing Jupyter notebook from validation.ipynb


In [ ]:
# pull files from Github
!git clone https://github.com/gmshroff/mlfin2022.git

In [ ]:
%cd mlfin2022/algostrats

Import Strategy - replace with your own class or modify rulestrats

In [ ]:
# upload rulestrats.ipynb and kaggle.json
from google.colab import files
uploaded=files.upload()

In [ ]:
!mkdir /root/.kaggle
!mv ./kaggle.json /root/.kaggle/.
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d gmshroff/mlfin2022

In [ ]:
!unzip mlfin2022.zip

In [2]:
from rulestrats import MomStrat

importing Jupyter notebook from rulestrats.ipynb


In [3]:
momStrat=MomStrat()

In [ ]:
from ipynb.fs.defs.mlstrats import MLStrat,MLConfig,ConfigFields

In [ ]:
mlstrat=MLStrat()

Using BackFeed

In [4]:
data=pd.read_csv('./capvol100.csv')

In [5]:
tickers=list(data.iloc[0:10]['ticker'].values)

In [6]:
feed=BackFeed(tickers=tickers,nd=3,nw=1)

In [ ]:
import pickle

In [ ]:
# with open('../../temp_data/btfeed.pickle','wb') as f: pickle.dump(feed,f)

In [ ]:
with open('../../temp_data/btfeed.pickle','rb') as f: feed=pickle.load(f)

In [11]:
bt=Backtest(feed,tickers=feed.tickers,add_features=True,target=.05,stop=.01,txcost=0.001,
            loc_exit=True,scan=True,topk=3,deploy=False,save_dfs=False)

In [12]:
bt.run_all(tickers=feed.tickers,model=momStrat,verbose=False)

  0%|          | 0/3 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
bt.run_all(tickers=feed.tickers,model=mlstrat,verbose=False)

In [9]:
bt.returns

{'ULTRACEMCO.NS': 2.8958118338527,
 'KOTAKBANK.NS': -44.56765448662545,
 'NESTLEIND.NS': 4.364058247157555,
 'ADANITRANS.NS': -381.1005383827462,
 'MARUTI.NS': 2.211574417312878}

In [13]:
bt.results

{'ULTRACEMCO.NS': {'25-Oct-2022': {'tot': 2.8958118338527,
   'rew': [(11, -0.12497151166212365, 374),
    (13, 0.04488878587061719, 374),
    (15, 0.04022444201478287, 374),
    (17, 0.3388340869017102, 374),
    (19, 0.5209945318853259, 374),
    (21, 0.39293517649978993, 374),
    (23, 0.3442407996529199, 374),
    (25, 0.27770348590353944, 374),
    (27, 0.26376670483223147, 374),
    (29, 0.28621639050552034, 374),
    (37, 0.510978941448386, 374)],
   'acts': [(-1, 11),
    (-1, 13),
    (-1, 15),
    (-1, 17),
    (-1, 19),
    (-1, 21),
    (-1, 23),
    (-1, 25),
    (-1, 27),
    (-1, 29),
    (-1, 37)]}},
 'KOTAKBANK.NS': {'25-Oct-2022': {'tot': -44.56765448662545,
   'rew': [(6, -1.246694277215169, 32),
    (7, -1.114039787830843, 31),
    (8, -1.177118673592755, 28),
    (9, -1.2658284150680765, 28),
    (10, -1.3777968082724774, 28),
    (11, -1.3049155772157137, 28),
    (12, -1.3517806106568278, 28),
    (13, -1.3491756944985194, 28),
    (14, -1.1797263085623506, 28),


Using DataFeed

In [ ]:
DATAFILE='./augdata_03-Jun-2022.csv'
df=pd.read_csv(DATAFILE)

In [ ]:
dataFeed=DataFeed(tickers=list(df.ticker.unique()[0:10]),dfgiven=True,df=df)

In [ ]:
bt=Backtest(dataFeed,tickers=dataFeed.tickers,target=.05,stop=.01,txcost=0.001,
            loc_exit=True,scan=True,topk=3,deploy=False)

In [ ]:
bt.run_all(tickers=dataFeed.tickers,model=momStrat)

In [ ]:
bt.results

In [ ]:
bt.returns

Getting additional data

In [ ]:
from featfuncs import add_global_indices_feed

In [ ]:
add_global_indices_feed(feed)

In [ ]:
feed.gdata

Validating remote strategy functions

In [ ]:
import anvil.server
anvil.server.connect("STKYCIXUMGFV2CURGMCMOFUV-MBK5ZXFC4MCKYKOE")

In [ ]:
# Note - first do a backtest using a local strategy; this will also update the feed with
# required features. Only then do remote validation. Else do the following first:
from featfuncs import add_addl_features_feed,add_sym_feature_feed
add_addl_features_feed(feed,tickers=feed.tickers)
add_sym_feature_feed(feed,tickers=feed.tickers)

In [ ]:
dfD={feed.tickers[i]:feed.getDataN(feed.tickers[i],feed.dates[0]) for i in range(4)}

In [ ]:
rpcname='s0_at_a_dot_edu'

In [ ]:
retval=Validate.remote_check_entry_batch(dfD,rpcname)

In [ ]:
list(momStrat.check_entry_batch(dfD))

In [ ]:
Validate(verbose=False).validate(retval=list(momStrat.check_entry_batch(dfD)),dfD=dfD)

In [ ]:
retval=Validate(feed).validate(retval=retval,dfD=dfD)

In [ ]:
retval1=retval.copy()
retval1[0]={'MARUTI.NS': 'five', 'KOTAKBANK.NS': 5, 'NESTLEIND.NS': 5, 'COALINDIA.NS': 5}

In [ ]:
Validate(feed).validate(retval=retval,dfD=dfD)

Backtesting remote strategy

In [ ]:
import anvil.server
anvil.server.connect("STKYCIXUMGFV2CURGMCMOFUV-MBK5ZXFC4MCKYKOE")

In [ ]:
bt=Backtest(feed,tickers=feed.tickers,add_features=True,target=.05,stop=.01,txcost=0.001,
            loc_exit=True,scan=True,topk=3,remote=True,
            data_cols=momStrat.data_cols,rpcname='s0_at_a_dot_edu')

In [ ]:
bt.run_all(tickers=feed.tickers)

In [ ]:
bt.results

In [ ]:
bt.returns

Experiments